In [ ]:
#=nb

In [31]:
!pip install kaggle

In [2]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import math

In [3]:
divice = ("cuda"
   if torch.cuda.is_available()
   else "mps"
   if torch.backends.mps.is_available()
   else "cpu")
divice

'cuda'

In [24]:
class vgg(nn.Module):
  def __init__(self, layer):
    super().__init__()
    self.layer = layer
    self.flatten = nn.Flatten()
    self.make_layer()
    dummy = torch.zeros(3, 224,224)
    o = self.flatten(self.Conv(dummy)).shape[1]
    self.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(0, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, 1000)
    )
    self.init_weight()
  def make_layer(self):
    self.Conv = nn.Sequential()
    prev = 3
    for i in self.layer:
      if i == "M":
        self.Conv.append( nn.MaxPool2d(kernel_size = 2 , stride = 2))
      elif i == "lrn":
        self.Conv.append( nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2)
         )
      else:
        self.Conv.append(nn.Conv2d(in_channels = prev, out_channels = i , kernel_size = 3, stride = 1, padding = 1))
        prev = i
  def init_weight(self):
    count = 1
    for i in self.Conv:
      if isinstance(i, nn.Conv2d):
        nn.init.xavier_normal_(i.weight)
        count += 1
      if count >= 4:
        break
    nn.init.xavier_normal_(self.fc[1].weight, gain = math.sqrt(2))
    nn.init.xavier_normal_(self.fc[4].weight, gain = math.sqrt(2))
    nn.init.xavier_normal_(self.fc[6].weight, gain = math.sqrt(2))
  def forward(self, X):
    X = self.Conv(X)
    X = self.flatten(X)
    X = self.fc(X)
    return X


In [27]:
a = [64,"M",128, "M", 256,256,"M", 512,512, "M", 512,512,"M"]
a_lrn = [64, "lrn", "M",128, "M",256,256, "M", 512,512, "M", 512,512, "M"]
b = [64, 64, "M", 128,128, "M", 256, 256, "M", 512, 512, "M", 512,512, "M"]
d = [64,64,"M",128,128,"M",256,256,256,"M", 512,512,512,"M", 512,512,512, "M"]
e = [64,64,"M",128,128,"M", 256,256,256,256, "M", 512,512,512,512, "M", 512,512,512,512, "M"]

In [29]:
vgg_11 = vgg(a).to(divice)
vgg_11_lrn = vgg(a_lrn).to(divice)
vgg_13 = vgg(b).to(divice)
vgg_16 = vgg(d).to(divice)
vgg_19 = vgg(e).to(divice)